# LLaVA

- [LLaVA: Large Language and Vision Assistant][1]
- [haotian-liu/LLaVA][2]
- [liuhaotian/LLaVA-Instruct-150K][3]

[1]: https://llava-vl.github.io/
[2]: https://github.com/haotian-liu/LLaVA
[3]: https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K

## 概要

LLaVA（Large Language and Vision Assistant）は、大規模マルチモーダルモデル（LMM）

視覚エンコーダー（Vision Encoder）と大規模言語モデル（LLM）を組み合わせた構造

テキスト版のGPT-4で、テキストと画像から成る指示データセットを作り、訓練（Visual Instruction Tuning）

未知の画像や指示に対して、マルチモーダル版のGPT-4に近い性能を実現（85.1%相対スコア）

## Visutal Instructionデータの作成

利用可能なマルチモーダル指示追従データセットは少なく、クラウドソーシングも属人的で難しい

データの収集にGPT-4を活用し、[COCO（Common Object in Context）][1]画像からデータを生成:

![](image/table1.png)

[1]: https://cocodataset.org/#explore

- 上部: テキスト版GPT-4に入力する2種類の記号表現
    1. キャプション: 視覚的なシーンの描写
    2. バウンディングボックス: シーン内の物体の概念と位置
- 下部: 記号表現から生成した3種類の指示追従データ（158K）
    1. 会話（58K）
    2. 詳細な記述（23K）
    3. 複雑な推論（77K）

## アーキテクチャ

事前学習済み言語モデル（Language Model）と視覚モデル（Vision Encoder）を活用:

![](image/fig1.png)

- $X_v$: 視覚入力
- $Z_v$: 視覚特徴量
- $H_v$: 視覚トークン
- $X_q$: 言語指示
- $H_q$: 言語トークン
- Language Model $f_\phi$
    - パラメータが$\phi$である言語モデル
    - 指示追従能力の高い公開モデルである[Vicuna][1]を使用
- Vision Encoder $g$
    - 事前学習済みCLIP視覚エンコーダViT-L/14を使用
    - 画像を視覚特徴量に変換 $Z_v = g(X_v)$

- $W$
    - 射影行列
    - 視覚特徴量を視覚トークンに変換 $H_v = W\cdot Z_v$
    - 視覚トークンを言語トークンと同様に扱う

[1]: https://lmsys.org/blog/2023-03-30-vicuna/

## 訓練

指示チューニングデータセットの構成:

$$
X_{\text{instruct}}^{t} = \begin{cases} \text{Randomly choose } [X_{q}^{1}, X_{v}] \text{ or } [X_{v}, X_{q}^{1}], & \text{the first turn } t=1 \\ X_{q}^{t}, & \text{the remaining turns } t>1 \end{cases}
$$

- 最初のターン（$t=1$）は2種類をランダムに組み合わせる
    1. 最初の質問$X_q^1$と画像$X_v$
    2. 画像$X_v$と最初の質問$X_q^1$
- それ以降のターン（$t \gt 1$）は、質問$X_q^t$だけ

指示チューニングデータセットの形式:

![](image/table2.png)

- システムメッセージ$X_{\text{system-message}}$を追加し、フォーマットに沿わせる
- 訓練時の損失は、アシスタントの回答$X_a^t$と終了トークン$<\text{STOP}>$に対して計算する（緑）

訓練は、モデルが正しい回答を生成できる確率を最大化するように学習する:

$$
p(X_{a} \mid X_{v}, X_{\text{instruct}}) = \prod_{i=1}^{L} p_{\theta}(x_{i} \mid X_{v}, X_{\text{instruct}, <i}, X_{a, <i})
$$

- $p(X_a | X_v, X_{\text{instruct}})$
    - 画像$X_v$とプロンプト$X_\text{instruct}$が与えられたときの正しい回答$X_a$を生成できる確率
- $\theta$: 学習可能なパラメータ
- $X_v$: 入力画像
- $X_{\text{instruct}\lt i}$: 現在の予測トークン$X_i$以前のプロンプト
- $X_{a\lt i}$: 現在の予測トークン$X_i$以前の回答

Instruction Tuningは2段階で行う:

1. 特徴量アラインメントのための事前学習
    - [CC3M][1]データセットをフィルタリングし、595Kの画像テキストペアにフィルタリング
        - 自然言語ライブラリSpacyを使い全キャプションから名詞句を抽出し、出現頻度をカウント
        - 頻度が3未満はスキップ
        - 頻度が100を超える場合は、サブセットを選択し制限
    - フィルタリングしたデータを、シングルターンの指示チューニングデータに変換
        - GPT-4が作成した単純な指示プロンプトと元のキャプションのデータ
    - 視覚エンコーダーと言語モデルの重みを凍結し、射影行列のみを訓練（$\theta = W$）
    - 画像特徴量$H_v$と言語特徴量$H_q$のアラインメントをとる
2. エンドツーエンドのファインチューニング
    - 射影行列と言語モデルの両方を訓練（$\theta = {W, \phi}$）
    - 2種類の手法で実験
        1. チャットボット用
            - COCOで作成した158Kの指示チューニングデータを使用
            - マルチターンの会話、シングルターンの詳細な記述・複雑な推論
            - 3種類のデータを均一にサンプリング
        2. 科学のクイズ用
            - [ScienceQA](https://scienceqa.github.io)ベンチマークのデータセットを使用
            - 質問に対し、テキストと画像でコンテキストを与え、複数の選択肢から回答を選択させる
            - シングルターンの会話形式

[1]: https://huggingface.co/datasets/pixparse/cc3m-wds

## 実験

Vicunaのハイパーパラメータに従い、8枚のA100 GPUで訓練

事前学習設定:

- CC-595Kデータセットを使用
- 学習率 2e-3
- バッチサイズ 128
- 1エポック

ファインチューニング:

- LLaVA-Instruct-158Kデータセット
- 学習率2e-5
- バッチサイズ32
- 3エポック

LLaVAは、単なるシーンの説明ではなく指示に正確に従った回答が可能:

![](image/table3.png)

GPT-4を使用して回答の品質を定量的に評価し、訓練フェーズごとに比較:

![](image/table4.png)

- 事前学習（Instruction Tuning）で、指示追従能力が50点以上改善（73.8）
- ファインチューニングで、推論能力が改善し更に7点改善（85.1）

OpenFlamingoやBLIP-2と比べると著しく高い性能:

![](image/table5.png)

LLaVAが苦手な質問:

![](image/table6.png)

- 写真からレストランの名前を当てる
- 写真からヨーグルトのブランド名を当てる
- ヨーグルトとイチゴが別にあるのにイチゴ味のヨーグルトがあると答えてしまう

科学の問題については、SOTAに近い性能（90.92%）:

![](image/table7.png)

- 訓練データ 12726件、検証データ4,241件、テストデータ 4241件
- 12エポックで訓練
- モデルに理由を予測させて、回答させる


## LLaVA 1.5

LLaVA 1.5は、オリジナルのLLaVAに少し修正を加えて性能を大幅に向上させたモデル:

1. 視覚と言語をつなぐコネクタ（Vision-Language Connector）を変更
    - 1つの全結合層から、2層のMLP（全結合層->GELU->全結合層）
2. 入力画像解像度を拡大（224x224から336x336）
  - パッチは14px四方なので、14*14=576に+1（CLSトークン）が画像特徴量の長さ
3. 学習データの増強（学術タスクのデータを追加）
    - 事前学習（8台A100で13Bを5.5時間、7Bを3.5時間）
        - [CC-3M Cocept-balanced 595K](https://huggingface.co/datasets/liuhaotian/LLaVA-CC3M-Pretrain-595K)
            - [CC-3M](https://github.com/google-research-datasets/conceptual-captions)データセットをフィルターし、BLIPでキャプションを構成キャプションを構成
        - [LAION/CC/SBU BLIP-Caption Concept-balanced 558K](https://huggingface.co/datasets/liuhaotian/LLaVA-Pretrain)
          - LAION-CC-SBUのサブセットでBLIPを使ってキャプションを高品質化
    - ファインチューニング（8台A100で13Bを20時間、7Bで10時間、LORAでメモリ削減可能）
        - [LLaVA-Instruct-150K](https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K)
            - GPT-4を用いて生成したマルチモーダルな指示追従データ（[プロンプト](https://github.com/haotian-liu/LLaVA/tree/main/playground/data/prompts)）
        - [COCO train2017](http://images.cocodataset.org/zips/train2017.zip)
            - 物体検出用のデータセット
        - [GQA](https://cs.stanford.edu/people/dorarad/gqa/)
            - [ダウンロード](https://downloads.cs.stanford.edu/nlp/data/gqa/images.zip)
            - 構成的な推論タスクのデータセット
        - [OCR-VQA](https://huggingface.co/datasets/howard-hou/OCR-VQA)
            - [ダウンロード](https://drive.google.com/drive/folders/1_GYPY5UkUy7HIcR0zq3ZCFgeZN7BAfm_?usp=sharing)
            - 画像中の文字の質問応答
        - TextVQA
            - [JSON](https://dl.fbaipublicfiles.com/textvqa/data/TextVQA_0.5.1_val.json)
            - [画像](https://dl.fbaipublicfiles.com/textvqa/images/train_val_images.zip)
            - 画像中のテキストの読み取りと推論
        - [VisualGenome](https://arxiv.org/abs/1602.07332)
            - 画像の詳細な領域説明
            - [part1](https://cs.stanford.edu/people/rak248/VG_100K_2/images.zip)
            - [part2](https://cs.stanford.edu/people/rak248/VG_100K_2/images2.zip)
4. 出力形式を指定するプロンプトを導入（短い回答と自然な回答を使い分けることが可能に）
    - "Answer the question using single word or phrase."など

## 実装

- [llava@1.2.2.post1][1]

[1]: https://github.com/haotian-liu/LLaVA/releases/tag/v1.2.0

In [ ]:
import logging
import os
import subprocess
import sys

if os.path.exists("debug.log"):
    os.remove("debug.log")

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler("debug.log")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.setLevel(logging.DEBUG)

NVIDIA_SMI = subprocess.run(["nvidia-smi"], capture_output=True, text=True).stdout
logging.info(NVIDIA_SMI)
logging.info(f"Python {sys.version}")

In [ ]:
%pip install -q bitsandbytes sentencepiece protobuf

from abc import ABC, abstractmethod
from enum import auto, Enum
from io import BytesIO
from PIL import Image
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, CLIPVisionModel, CLIPImageProcessor, CLIPVisionConfig, LlamaConfig, LlamaModel, LlamaForCausalLM
from transformers.generation.utils import GenerateOutput
from transformers.modeling_outputs import CausalLMOutputWithPast
from typing import List, Optional, Tuple, Union
import base64
import dataclasses
import os
import re
import requests
import shutil
import torch
import torch.nn as nn
import transformers
import warnings

logger.info(f"Transformers: {transformers.__version__}")

### Vision Tower

In [ ]:
class CLIPVisionTower(nn.Module):
    """
    画像をエンコードする視覚エンコーダCLIPのラッパークラス
    build_vision_tower関数から呼び出される
    LlavaMetaModelクラスで実体化される
    """

    def __init__(self, vision_tower, args, delay_load=False):
        logger.info(f"CLIPVisionTowerを初期化 {vision_tower=} {args=} {delay_load=}")

        super().__init__()

        self.is_loaded = False

        # openai/clip-vit-large-patch14-336
        # https://huggingface.co/openai/clip-vit-large-patch14-336
        self.vision_tower_name = vision_tower

        # CLIPの隠れ状態から最後からN番目の層を使用する
        # -2（最後から2番目の層）
        self.select_layer = args.mm_vision_select_layer

        # 隠れ状態から選択する特徴量の種類
        # patch（パッチ特徴量のみ）
        self.select_feature = getattr(args, 'mm_vision_select_feature', 'patch')

        # False
        if not delay_load:
            self.load_model()

        # False
        elif getattr(args, 'unfreeze_mm_vision_tower', False):
            self.load_model()

        # True
        else:
            # CLIPVisionConfigを読み込む
            # https://huggingface.co/openai/clip-vit-large-patch14-336/blob/main/config.json
            self.cfg_only = CLIPVisionConfig.from_pretrained(self.vision_tower_name)
            logger.debug(f"CLIPVisionConfigを読み込む {self.cfg_only=}")

    def load_model(self, device_map=None):
        """
        CLIPImageProcessorとCLIPVisionModelをロードする
        load_pretrained_model関数から呼び出される
        """
        logger.info(f"CLIPVisionTowerを読み込み {device_map=}")

        # 1) 入力値の検証

        if self.is_loaded:
            print('{} is already loaded, `load_model` called again, skipping.'.format(self.vision_tower_name))
            return

        # 2) CLIPImageProcessorを読み込み

        self.image_processor = CLIPImageProcessor.from_pretrained(
            self.vision_tower_name
        )

        logger.debug(f"CLIPImageProcessorをロード {self.image_processor=}")

        # 3) CLIPVisionModelを読み込み

        self.vision_tower = CLIPVisionModel.from_pretrained(
            self.vision_tower_name,
            device_map=device_map
        )

        logger.debug(f"CLIPVisionModelをロード {self.vision_tower=}")

        # 4) 後処理

        # ビジョンタワーのパラメータを固定
        self.vision_tower.requires_grad_(False)

        # 読み込み済みフラグを有効化
        self.is_loaded = True

    def feature_select(self, image_forward_outs):
        """
        順伝播直後の隠れ状態から特徴量を選択する
        forward関数から呼び出される

        Args:
            image_forward_outs: CLIPVisionModelの順伝搬出力
        Returns:
            image_features: 選択された特徴量
        """
        logger.info(f"CLIPVisionTowerの特徴量選択 {type(image_forward_outs)=}")

        image_features = image_forward_outs.hidden_states[self.select_layer]
        logger.debug(f"選択された層の隠れ状態 {image_features.shape=}")

        # True
        if self.select_feature == 'patch':
            # 0番目はCLSトークンなので除外
            image_features = image_features[:, 1:]
            logger.debug(f"パッチ特徴量のみ選択 {image_features.shape=}")

        # False
        elif self.select_feature == 'cls_patch':
            image_features = image_features

        # False
        else:
            raise ValueError(f'Unexpected select feature: {self.select_feature}')

        return image_features

    @torch.no_grad()
    def forward(self, images):
        logger.info(f"CLIPVisionTowerの順伝搬 {type(images)=}")

        # 画像がリストの場合
        # False
        if type(images) is list:
            image_features = []
            for image in images:
                image_forward_out = self.vision_tower(image.to(device=self.device, dtype=self.dtype).unsqueeze(0), output_hidden_states=True)
                image_feature = self.feature_select(image_forward_out).to(image.dtype)
                image_features.append(image_feature)

        # 画像が1枚の場合
        else:

            # GPUに転送して順伝搬し、隠れ状態を出力
            image_forward_outs = self.vision_tower(
                images.to(device=self.device, dtype=self.dtype),
                output_hidden_states=True
            )

            # 隠れ状態から特徴量を選択
            image_features = self.feature_select(image_forward_outs).to(images.dtype)

        return image_features

    @property
    def dummy_feature(self):
        return torch.zeros(1, self.hidden_size, device=self.device, dtype=self.dtype)

    @property
    def dtype(self):
        return self.vision_tower.dtype

    @property
    def device(self):
        return self.vision_tower.device

    @property
    def config(self):
        if self.is_loaded:
            return self.vision_tower.config
        else:
            return self.cfg_only

    @property
    def hidden_size(self):
        return self.config.hidden_size

    @property
    def num_patches_per_side(self):
        return self.config.image_size // self.config.patch_size

    @property
    def num_patches(self):
        return (self.config.image_size // self.config.patch_size) ** 2

In [ ]:
def build_vision_tower(vision_tower_cfg, **kwargs):
    """
    ビジョンタワーを実体化するヘルパー関数
    LlavaMetaModelから呼び出される

    Args:
        vision_tower_cfg: ビジョンタワーの設定オブジェクト
        **kwargs: その他の引数
    Returns:
        構築されたビジョンタワーのインスタンス
    """
    logger.info(f"ビジョンタワーを構築 {vision_tower_cfg=}, {kwargs=}")

    # 1) メタデータを抽出

    # ビジョンタワーの名前を取得
    # openai/clip-vit-large-patch14-336
    # https://huggingface.co/liuhaotian/llava-v1.5-7b/blob/main/config.json
    vision_tower = getattr(
        vision_tower_cfg,
        'mm_vision_tower',
        getattr(vision_tower_cfg, 'vision_tower', None)
    )

    is_absolute_path_exists = os.path.exists(vision_tower)
    logger.debug(f"{is_absolute_path_exists=}")

    # ClipVisionTowerS2クラスを使用する場合（最新版のClipVisionTower）
    # False
    use_s2 = getattr(vision_tower_cfg, 's2', False)
    logger.debug(f"{use_s2=}")

    # 2) ビジョンタワーの実体化

    # True
    if is_absolute_path_exists or \
        vision_tower.startswith("openai") or \
        vision_tower.startswith("laion") or \
        "ShareGPT4V" in vision_tower:

        # False
        if use_s2:
            return CLIPVisionTowerS2(vision_tower, args=vision_tower_cfg, **kwargs)
        else:
            # CLIPVisionTowerをインスタンス化
            return CLIPVisionTower(vision_tower, args=vision_tower_cfg, **kwargs)

    raise ValueError(f'Unknown vision tower: {vision_tower}')

### Vision Projector

In [ ]:
def build_vision_projector(config, delay_load=False, **kwargs):
    """
    ビジョンプロジェクターを構築するユーティリティ関数
    LlavaMetaModelから呼び出される
    今回はGELU活性化関数を搭載した2層の全結合層からなるMLPを使用
    ビジョンタワーの出力特徴量を視覚トークンに変換する

    Args:
        config: モデルの設定オブジェクト
        delay_load: プロジェクターの遅延ロードフラグ
        **kwargs: その他の引数
    Returns:
        構築されたビジョンプロジェクターのインスタンス
    """
    logger.info(f"ビジョンプロジェクターを構築 {config=}, {delay_load=}, {kwargs=}")

    # 1) メタデータの抽出

    # https://huggingface.co/liuhaotian/llava-v1.5-7b/blob/main/config.json
    # mlp2x_gelu: GELU付きの2層MLP
    projector_type = getattr(config, 'mm_projector_type', 'linear')
    logger.debug(f"{projector_type=}")

    # False
    if projector_type == 'linear':
        print("Using linear projector for vision features.")
        return nn.Linear(config.mm_hidden_size, config.hidden_size)

    # mlp2x_gelu -> 2（2層の全結合層からなるMLP）
    mlp_gelu_match = re.match(r'^mlp(\d+)x_gelu$', projector_type)

    # True
    if mlp_gelu_match:

        # 2
        mlp_depth = int(mlp_gelu_match.group(1))
        logger.debug(f"{mlp_depth=}")

        modules = [
            # 1つ目の全結合層
            # 1024 -> 4096
            nn.Linear(config.mm_hidden_size, config.hidden_size)
        ]

        # 2層なので1回ループ
        for _ in range(1, mlp_depth):

            # GELU活性化関数
            modules.append(nn.GELU())

            # 2つ目の全結合層
            # 4096 -> 4096
            modules.append(nn.Linear(config.hidden_size, config.hidden_size))

        return nn.Sequential(*modules)

    if projector_type == 'identity':
        return IdentityMap()

    raise ValueError(f'Unknown projector type: {projector_type}')

### LlavaMetaModel

In [ ]:
class LlavaMetaModel:
    """
    ビジョンタワーとビジョンプロジェクターをラップするクラス
    画像の入力をビジョンタワーで画像特徴量に変換し、ビジョンプロジェクターで視覚トークンに変換する
    LlavaLlamaModelが継承し、言語モデル（LlamaModel）と統合される
    """

    def __init__(self, config):
        logger.info(f"LlavaMetaModelを初期化 {config=}")

        # https://huggingface.co/liuhaotian/llava-v1.5-7b/blob/main/config.json
        super(LlavaMetaModel, self).__init__(config)

        # True
        if hasattr(config, "mm_vision_tower"):

            # ビジョンタワーの構築
            self.vision_tower = build_vision_tower(config, delay_load=True)

            # ビジョンプロジェクターの構築
            self.mm_projector = build_vision_projector(config)

            # False
            if 'unpad' in getattr(config, 'mm_patch_merge_type', ''):
                self.image_newline = nn.Parameter(
                    torch.empty(config.hidden_size, dtype=self.dtype)
                )

    def get_vision_tower(self):
        """
        ビジョンタワーを取得するヘルパー関数
        初期化や画像特徴量抽出時に使用される

        Returns:
            ビジョンタワーのインスタンス
        """
        vision_tower = getattr(self, 'vision_tower', None)
        if type(vision_tower) is list:
            vision_tower = vision_tower[0]
        return vision_tower

    def initialize_vision_modules(self, model_args, fsdp=None):
        """
        使わない
        """
        logger.info(f"ビジョンモジュールを初期化 {model_args=}, {fsdp=}")
        vision_tower = model_args.vision_tower
        mm_vision_select_layer = model_args.mm_vision_select_layer
        mm_vision_select_feature = model_args.mm_vision_select_feature
        pretrain_mm_mlp_adapter = model_args.pretrain_mm_mlp_adapter
        mm_patch_merge_type = model_args.mm_patch_merge_type

        self.config.mm_vision_tower = vision_tower

        if self.get_vision_tower() is None:
            vision_tower = build_vision_tower(model_args)

            if fsdp is not None and len(fsdp) > 0:
                self.vision_tower = [vision_tower]
            else:
                self.vision_tower = vision_tower
        else:
            if fsdp is not None and len(fsdp) > 0:
                vision_tower = self.vision_tower[0]
            else:
                vision_tower = self.vision_tower
            vision_tower.load_model()

        self.config.use_mm_proj = True
        self.config.mm_projector_type = getattr(model_args, 'mm_projector_type', 'linear')
        self.config.mm_hidden_size = vision_tower.hidden_size
        self.config.mm_vision_select_layer = mm_vision_select_layer
        self.config.mm_vision_select_feature = mm_vision_select_feature
        self.config.mm_patch_merge_type = mm_patch_merge_type

        if getattr(self, 'mm_projector', None) is None:
            self.mm_projector = build_vision_projector(self.config)

            if 'unpad' in mm_patch_merge_type:
                embed_std = 1 / torch.sqrt(torch.tensor(self.config.hidden_size, dtype=self.dtype))
                self.image_newline = nn.Parameter(
                    torch.randn(self.config.hidden_size, dtype=self.dtype) * embed_std
                )
        else:
            # In case it is frozen by LoRA
            for p in self.mm_projector.parameters():
                p.requires_grad = True

        if pretrain_mm_mlp_adapter is not None:
            mm_projector_weights = torch.load(pretrain_mm_mlp_adapter, map_location='cpu')
            def get_w(weights, keyword):
                return {k.split(keyword + '.')[1]: v for k, v in weights.items() if keyword in k}

            self.mm_projector.load_state_dict(get_w(mm_projector_weights, 'mm_projector'))

### LlavaMetaForCausalLM

In [ ]:
class LlavaMetaForCausalLM(ABC):
    """
    マルチモーダル言語モデルの抽象基底クラス
    LlavaLlamaForCausalLMが継承し、LlavaMetaModelとLlamaForCausalLMを統合する
    """

    @abstractmethod
    def get_model(self):
        pass

    def get_vision_tower(self):
        """
        ビジョンタワーを取得するヘルパー関数
        prepare_inputs_labels_for_multimodal関数から呼び出される
        """
        return self.get_model().get_vision_tower()

    def encode_images(self, images):
        """
        画像をエンコードして特徴量を抽出する
        prepare_inputs_labels_for_multimodal関数から呼び出される

        Args:
            images: 画像データのテンソル
        Returns:
            image_features: 抽出された画像特徴量のテンソル
        """
        logger.info(f"画像をエンコード {images.shape=}")

        # 画像をビジョンタワーでエンコードして画像特徴量を取得
        image_features = self.get_model().get_vision_tower()(images)
        logger.debug(f"ビジョンタワーでエンコード {image_features.shape=}")

        # 画像特徴量をビジョンプロジェクターで視覚トークンに変換
        image_features = self.get_model().mm_projector(image_features)
        logger.debug(f"ビジョンプロジェクターで変換 {image_features.shape=}")

        return image_features

    def prepare_inputs_labels_for_multimodal(self, input_ids, position_ids, attention_mask, past_key_values, labels, images, image_sizes=None):
        """
        画像をエンコードし、マルチモーダル入力に必要なテンソルを準備する
        視覚トークンをプロンプトの<image>の位置に挿入し、対応するアテンションマスクとラベルを更新する
        LlavaLlamaForCausalLMの順伝播の最初（Prefill）と生成（Decode）の度に呼び出される

        Args:
            input_ids: テキストのトークンIDのテンソル
            position_ids: 位置IDのテンソル
            attention_mask: アテンションマスクのテンソル
            past_key_values: 過去のキーとバリューのテンソル
            labels: ラベルのテンソル
            images: 画像データのテンソルまたはリスト
            image_sizes: 画像サイズのリスト（オプション）
        Returns:
            input_ids: 更新されたテキストのトークンIDのテンソル
            position_ids: 更新された位置IDのテンソル
            attention_mask: 更新されたアテンションマスクのテンソル
            past_key_values: 更新された過去のキーとバリューのテンソル
            image_features: 抽出された画像特徴量のテンソル
            labels: 更新されたラベルのテンソル
        """
        logger.info(f"マルチモーダル入力の準備 {input_ids.shape=}, {position_ids.shape if position_ids is not None else None=}, {attention_mask.shape if attention_mask is not None else None=}, {past_key_values=}, {labels.shape if labels is not None else None=}, {type(images)=}, {image_sizes=}")

        # 1) 画像をエンコード

        # ビジョンタワーを取得
        vision_tower = self.get_vision_tower()

        # 画像がない場合
        # False
        if vision_tower is None or images is None or input_ids.shape[1] == 1:
            return input_ids, position_ids, attention_mask, past_key_values, None, labels

        # 画像が複数ある場合
        # False
        if type(images) is list or images.ndim == 5:

            if type(images) is list:
                images = [x.unsqueeze(0) if x.ndim == 3 else x for x in images]

            concat_images = torch.cat([image for image in images], dim=0)

            image_features = self.encode_images(concat_images)

            split_sizes = [image.shape[0] for image in images]

            image_features = torch.split(image_features, split_sizes, dim=0)

            mm_patch_merge_type = getattr(
                self.config,
                'mm_patch_merge_type',
                'flat'
            )

            image_aspect_ratio = getattr(
                self.config,
                'image_aspect_ratio',
                'square'
            )

            if mm_patch_merge_type == 'flat':
                image_features = [x.flatten(0, 1) for x in image_features]

            elif mm_patch_merge_type.startswith('spatial'):
                new_image_features = []
                for image_idx, image_feature in enumerate(image_features):
                    if image_feature.shape[0] > 1:
                        base_image_feature = image_feature[0]
                        image_feature = image_feature[1:]
                        height = width = self.get_vision_tower().num_patches_per_side
                        assert height * width == base_image_feature.shape[0]
                        if image_aspect_ratio == 'anyres':
                            num_patch_width, num_patch_height = get_anyres_image_grid_shape(image_sizes[image_idx], self.config.image_grid_pinpoints, self.get_vision_tower().config.image_size)
                            image_feature = image_feature.view(num_patch_height, num_patch_width, height, width, -1)
                        else:
                            raise NotImplementedError
                        if 'unpad' in mm_patch_merge_type:
                            image_feature = image_feature.permute(4, 0, 2, 1, 3).contiguous()
                            image_feature = image_feature.flatten(1, 2).flatten(2, 3)
                            image_feature = unpad_image(image_feature, image_sizes[image_idx])
                            image_feature = torch.cat((
                                image_feature,
                                self.model.image_newline[:, None, None].expand(*image_feature.shape[:-1], 1).to(image_feature.device)
                            ), dim=-1)
                            image_feature = image_feature.flatten(1, 2).transpose(0, 1)
                        else:
                            image_feature = image_feature.permute(0, 2, 1, 3, 4).contiguous()
                            image_feature = image_feature.flatten(0, 3)
                        image_feature = torch.cat((base_image_feature, image_feature), dim=0)
                    else:
                        image_feature = image_feature[0]
                        if 'unpad' in mm_patch_merge_type:
                            image_feature = torch.cat((
                                image_feature,
                                self.model.image_newline[None].to(image_feature.device)
                            ), dim=0)
                    new_image_features.append(image_feature)
                image_features = new_image_features
            else:
                raise ValueError(f"Unexpected mm_patch_merge_type: {self.config.mm_patch_merge_type}")

        # True
        else:
            logger.debug(f"単一の画像をエンコード {images.shape}")

            # (1, 3, 336, 336) -> (1, 576, 4096)
            image_features = self.encode_images(images)

        # False
        if getattr(self.config, 'tune_mm_mlp_adapter', False) and \
            getattr(self.config, 'mm_use_im_start_end', False):
            raise NotImplementedError

        # 2) ラベル・位置ID・アテンションマスクを必要に応じて初期化もしくは適用

        # 変更前の各変数を保存（Noneの場合はあとで捨てる）
        _labels = labels
        _position_ids = position_ids
        _attention_mask = attention_mask

        # 初回（prefill）はTrueで、2回目以降（decode）はFalse
        if attention_mask is None:
            # アテンションマスクを1で初期化
            # (1, 59)
            attention_mask = torch.ones_like(input_ids, dtype=torch.bool)
            logger.debug(f"{attention_mask=}")
        else:
            attention_mask = attention_mask.bool()

        # 初回（prefill）はTrueで、2回目以降（decode）はFalse
        if position_ids is None:
            # 位置IDを[0, 1, 2, ...]で初期化
            # (1, 59)
            position_ids = torch.arange(
                0,
                input_ids.shape[1],
                dtype=torch.long,
                device=input_ids.device)

        # 推論時は常にTrue
        if labels is None:
            # ラベルをIGNORE_INDEXで初期化
            # (1, 59)
            labels = torch.full_like(input_ids, IGNORE_INDEX)

        # 変更前の入力IDを保存
        _input_ids = input_ids

        # アテンションマスクを使用して入力のパディングを削除
        input_ids = [
            cur_input_ids[cur_attention_mask] \
            for cur_input_ids, cur_attention_mask \
            in zip(input_ids, attention_mask)
        ]

        # アテンションマスクを使用してラベルのパディングを削除
        labels = [
            cur_labels[cur_attention_mask] \
            for cur_labels, cur_attention_mask \
            in zip(labels, attention_mask)
        ]

        # 3) 入力シーケンスの画像トークンを視覚トークンに置換

        new_input_embeds = []
        new_labels = []
        cur_image_idx = 0

        # 入力シーケンスごとに処理
        for batch_idx, cur_input_ids in enumerate(input_ids):
            logger.debug(f"処理中のバッチ {batch_idx=}, {cur_input_ids.shape=}")

            # 画像トークンの数をカウント
            # 1
            num_images = (cur_input_ids == IMAGE_TOKEN_INDEX).sum()
            logger.debug(f"{batch_idx=}, {num_images=}")

            # 画像が含まれない場合の処理
            # False
            if num_images == 0:
                cur_image_features = image_features[cur_image_idx]
                cur_input_embeds_1 = self.get_model().embed_tokens(cur_input_ids)
                cur_input_embeds = torch.cat(
                    [cur_input_embeds_1, cur_image_features[0:0]
                ], dim=0)
                new_input_embeds.append(cur_input_embeds)
                new_labels.append(labels[batch_idx])
                cur_image_idx += 1
                continue

            # 画像が含まれる場合の処理

            # [-1, 35, 59]
            image_token_indices = [-1] + \
                torch.where(cur_input_ids == IMAGE_TOKEN_INDEX)[0].tolist() + \
                [cur_input_ids.shape[0]]
            logger.debug(f"{batch_idx=}, {image_token_indices=}")

            cur_input_ids_noim = []
            cur_labels = labels[batch_idx]
            cur_labels_noim = []

            # 画像トークンのインデックスごとにループ
            for i in range(len(image_token_indices) - 1):

                # 画像トークンを除いた入力シーケンスをリストに追加
                cur_input_ids_noim.append(
                    cur_input_ids[image_token_indices[i]+1:image_token_indices[i+1]]
                )

                # 画像トークンを除いたラベルをリストに追加
                cur_labels_noim.append(
                    cur_labels[image_token_indices[i]+1:image_token_indices[i+1]]
                )

            # 画像トークンを除いた各シーケンスのサイズを取得
            split_sizes = [x.shape[0] for x in cur_labels_noim]

            # 画像トークンを除いた入力シーケンスに埋め込みを適用
            # (58, 4096)
            cur_input_embeds = self.get_model().embed_tokens(
                torch.cat(cur_input_ids_noim)
            )
            logger.debug(f"{cur_input_embeds.shape=}")

            # 画像トークンを除いた入力埋め込みを分割
            # 2
            cur_input_embeds_no_im = torch.split(
                cur_input_embeds,
                split_sizes,
                dim=0
            )
            logger.debug(f"{len(cur_input_embeds_no_im)=}")

            cur_new_input_embeds = []
            cur_new_labels = []

            # 画像の枚数+1回ループ
            for i in range(num_images + 1):

                # テキスト部分の埋め込みを追加
                cur_new_input_embeds.append(cur_input_embeds_no_im[i])

                # テキスト部分のラベルを追加
                cur_new_labels.append(cur_labels_noim[i])

                # 画像1枚で最初のループの場合
                if i < num_images:

                    # 視覚トークンを追加
                    cur_image_features = image_features[cur_image_idx]
                    cur_image_idx += 1
                    cur_new_input_embeds.append(cur_image_features)

                    # 画像部分に対応する正解ラベルはIGNORE_INDEX（-100)
                    cur_new_labels.append(
                        torch.full((cur_image_features.shape[0],),
                        IGNORE_INDEX,
                        device=cur_labels.device,
                        dtype=cur_labels.dtype))

            # GPUに転送
            cur_new_input_embeds = [x.to(self.device) for x in cur_new_input_embeds]

            # 更新された入力シーケンスを結合
            cur_new_input_embeds = torch.cat(cur_new_input_embeds)

            # 更新されたラベルを結合
            cur_new_labels = torch.cat(cur_new_labels)

            # リストに追加
            new_input_embeds.append(cur_new_input_embeds)

            # リストに追加
            new_labels.append(cur_new_labels)

        # 4) 視覚トークンを挿入した後のパディングの調整

        # トークナイザーの最大長を取得
        # None
        tokenizer_model_max_length = getattr(
            self.config,
            'tokenizer_model_max_length',
            None)
        logger.debug(f"{tokenizer_model_max_length=}")

        # 最大長がある場合
        # False
        if tokenizer_model_max_length is not None:
            new_input_embeds = [x[:tokenizer_model_max_length] for x in new_input_embeds]
            new_labels = [x[:tokenizer_model_max_length] for x in new_labels]

        # 各シーケンスの最大長を計算
        max_len = max(x.shape[0] for x in new_input_embeds)
        logger.debug(f"{max_len=}")

        # バッチサイズを取得
        batch_size = len(new_input_embeds)
        logger.debug(f"{batch_size=}")

        new_input_embeds_padded = []

        # ラベルのパディングを-100で初期化
        new_labels_padded = torch.full(
            (batch_size, max_len),
            IGNORE_INDEX,
            dtype=new_labels[0].dtype,
            device=new_labels[0].device)

        # アテンションマスクを0で初期化
        attention_mask = torch.zeros(
            (batch_size, max_len),
            dtype=attention_mask.dtype,
            device=attention_mask.device)

        # 位置IDを0で初期化
        position_ids = torch.zeros(
            (batch_size, max_len),
            dtype=position_ids.dtype,
            device=position_ids.device)

        # 各シーケンスとラベルでループ
        for i, (cur_new_embed, cur_new_labels) in enumerate(zip(new_input_embeds, new_labels)):

            # 現在のシーケンス長を取得
            cur_len = cur_new_embed.shape[0]
            logger.debug(f"{i=}, {cur_len=}")

            # 左パディングの場合
            # False
            if getattr(self.config, 'tokenizer_padding_side', 'right') == "left":
                logger.debug(f"左パディング {i=}, {cur_len=}")
                new_input_embeds_padded.append(torch.cat((
                    torch.zeros((max_len - cur_len, cur_new_embed.shape[1]), dtype=cur_new_embed.dtype, device=cur_new_embed.device),
                    cur_new_embed
                ), dim=0))
                if cur_len > 0:
                    new_labels_padded[i, -cur_len:] = cur_new_labels
                    attention_mask[i, -cur_len:] = True
                    position_ids[i, -cur_len:] = torch.arange(0, cur_len, dtype=position_ids.dtype, device=position_ids.device)

            # 右パディングの場合
            # True
            else:
                logger.debug(f"右パディング {i=}, {cur_len=}")

                # 右側をゼロでパディング
                new_input_embeds_padded.append(torch.cat((
                    cur_new_embed,
                    torch.zeros((max_len - cur_len, cur_new_embed.shape[1]),
                    dtype=cur_new_embed.dtype,
                    device=cur_new_embed.device)
                ), dim=0))

                # 現在のシーケンス長が0より大きい場合
                if cur_len > 0:
                    # パディングのテンソルを上書き
                    new_labels_padded[i, :cur_len] = cur_new_labels

                    # アテンションマスクを上書き
                    attention_mask[i, :cur_len] = True

                    # 位置IDを上書き
                    position_ids[i, :cur_len] = torch.arange(
                        0,
                        cur_len,
                        dtype=position_ids.dtype,
                        device=position_ids.device)

        # リストをテンソルに変換
        new_input_embeds = torch.stack(new_input_embeds_padded, dim=0)
        logger.debug(f"パディング後の入力埋め込み {new_input_embeds.shape=}")

        # 5) 出力を整形

        # ラベルがもともとない場合はNoneに戻す
        if _labels is None:
            # Noneに戻す
            new_labels = None
        else:
            # 出力を新しいラベルに置換
            new_labels = new_labels_padded

        # アテンションマスクが元々無い場合
        if _attention_mask is None:
            # Noneに戻す
            attention_mask = None
        else:
            # 出力を新しいアテンションマスクに置換
            attention_mask = attention_mask.to(dtype=_attention_mask.dtype)

        # 位置IDが元々無い場合
        if _position_ids is None:
            # Noneに戻す
            position_ids = None
        
        logger.debug(f"マルチモーダル入力の準備完了 {new_input_embeds.shape=}, {position_ids=}, {attention_mask=}, {past_key_values=}, {new_labels=}")

        return None, position_ids, attention_mask, past_key_values, new_input_embeds, new_labels

    def initialize_vision_tokenizer(self, model_args, tokenizer):
        logger.info(f"ビジョントークナイザーの初期化 {model_args=}, {tokenizer=}")
        if model_args.mm_use_im_patch_token:
            tokenizer.add_tokens([DEFAULT_IMAGE_PATCH_TOKEN], special_tokens=True)
            self.resize_token_embeddings(len(tokenizer))

        if model_args.mm_use_im_start_end:
            num_new_tokens = tokenizer.add_tokens([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN], special_tokens=True)
            self.resize_token_embeddings(len(tokenizer))

            if num_new_tokens > 0:
                input_embeddings = self.get_input_embeddings().weight.data
                output_embeddings = self.get_output_embeddings().weight.data

                input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(
                    dim=0, keepdim=True)
                output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(
                    dim=0, keepdim=True)

                input_embeddings[-num_new_tokens:] = input_embeddings_avg
                output_embeddings[-num_new_tokens:] = output_embeddings_avg

            if model_args.tune_mm_mlp_adapter:
                for p in self.get_input_embeddings().parameters():
                    p.requires_grad = True
                for p in self.get_output_embeddings().parameters():
                    p.requires_grad = False

            if model_args.pretrain_mm_mlp_adapter:
                mm_projector_weights = torch.load(model_args.pretrain_mm_mlp_adapter, map_location='cpu')
                embed_tokens_weight = mm_projector_weights['model.embed_tokens.weight']
                assert num_new_tokens == 2
                if input_embeddings.shape == embed_tokens_weight.shape:
                    input_embeddings[-num_new_tokens:] = embed_tokens_weight[-num_new_tokens:]
                elif embed_tokens_weight.shape[0] == num_new_tokens:
                    input_embeddings[-num_new_tokens:] = embed_tokens_weight
                else:
                    raise ValueError(f"Unexpected embed_tokens_weight shape. Pretrained: {embed_tokens_weight.shape}. Current: {input_embeddings.shape}. Numer of new tokens: {num_new_tokens}.")
        elif model_args.mm_use_im_patch_token:
            if model_args.tune_mm_mlp_adapter:
                for p in self.get_input_embeddings().parameters():
                    p.requires_grad = False
                for p in self.get_output_embeddings().parameters():
                    p.requires_grad = False

### LlavaLlamaForCausalLM

In [ ]:
class LlavaConfig(LlamaConfig):
    """
    言語モデルの設定クラスをLLaVA用に拡張
    """

    model_type = "llava_llama"

In [ ]:
class LlavaLlamaModel(LlavaMetaModel, LlamaModel):
    """
    LlavaMetaModelとLlamaModelを統合するクラス
    LlavaLlamaForCausalLMでインスタンス化される
    """

    config_class = LlavaConfig

    def __init__(self, config: LlamaConfig):

        logger.info(f"LlavaLlamaModelを初期化 {config=}")

        super(LlavaLlamaModel, self).__init__(config)

In [ ]:
class LlavaLlamaForCausalLM(LlamaForCausalLM, LlavaMetaForCausalLM):
    config_class = LlavaConfig

    def __init__(self, config):
        logger.info(f"LlavaLlamaForCausalLMを初期化 {config=}")

        # https://huggingface.co/liuhaotian/llava-v1.5-7b/blob/main/config.json
        super(LlamaForCausalLM, self).__init__(config)

        # LlavaLlamaModelのインスタンス化
        self.model = LlavaLlamaModel(config)

        # Tensor Parallelismの設定
        # 1（TPを使用しない）
        self.pretraining_tp = config.pretraining_tp

        # 語彙数の設定
        # 32000
        self.vocab_size = config.vocab_size

        # 言語モデルの線形層を初期化
        # 4096 -> 32,000
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

        # 重みを初期化し、最終処理を適用
        self.post_init()

    def get_model(self):
        "LlavaLlamaModelのインスタンスを取得"
        return self.model

    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        images: Optional[torch.FloatTensor] = None,
        image_sizes: Optional[List[List[int]]] = None,
        return_dict: Optional[bool] = None,
        cache_position=None, # https://github.com/huggingface/transformers/issues/29426
    ) -> Union[Tuple, CausalLMOutputWithPast]:
        """
        今回は使わない
        """

        if inputs_embeds is None:
            (
                input_ids,
                position_ids,
                attention_mask,
                past_key_values,
                inputs_embeds,
                labels
            ) = self.prepare_inputs_labels_for_multimodal(
                input_ids,
                position_ids,
                attention_mask,
                past_key_values,
                labels,
                images,
                image_sizes
            )

        return super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            labels=labels,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

    @torch.no_grad()
    def generate(
        self,
        inputs: Optional[torch.Tensor] = None,
        images: Optional[torch.Tensor] = None,
        image_sizes: Optional[torch.Tensor] = None,
        **kwargs,
    ) -> Union[GenerateOutput, torch.LongTensor]:
        """
        画像とテキストを入力として受け取り、テキストを生成する
        eval_model関数から呼び出される

        Args:
            inputs: テキスト入力のテンソル
            images: 画像プロセッサーで前処理済みの画像入力のテンソル
            image_sizes: 元の画像サイズのリスト
            **kwargs:
                do_sample: サンプリングを行うかどうか
                temperature: サンプリングの温度パラメータ
                top_p: nucleusサンプリングの確率閾値
                num_beams: ビームサーチのビーム数
                max_new_tokens: 生成する最大トークン数
                use_cache: キャッシュを使用するかどうか
                ...
        """

        logger.info(f"テキスト生成開始 {inputs.shape=}, {images.shape=}, {image_sizes=}, {kwargs=}")

        # 1) パラメータの抽出

        # None
        position_ids = kwargs.pop("position_ids", None)

        # None
        attention_mask = kwargs.pop("attention_mask", None)

        # None
        if "inputs_embeds" in kwargs:
            raise NotImplementedError("`inputs_embeds` is not supported")

        # 2) マルチモーダル入力の準備

        # 画像がある場合
        # True
        if images is not None:

            # 画像をエンコードし、視覚トークンをプロンプトの<image>の位置に挿入
            # パッディングを調整し、それに応じてアテンションマスクと位置IDを更新
            (
                inputs,
                position_ids,
                attention_mask,
                _,
                inputs_embeds,
                _
            ) = self.prepare_inputs_labels_for_multimodal(
                inputs,
                position_ids,
                attention_mask,
                None,
                None,
                images,
                image_sizes=image_sizes
            )

        # False
        else:
            inputs_embeds = self.get_model().embed_tokens(inputs)

        # LlamaForCausalLMのgenerateメソッドを呼び出す
        result = super().generate(
            position_ids=position_ids,
            attention_mask=attention_mask,
            inputs_embeds=inputs_embeds,
            **kwargs
        )
        logger.debug(f"テキスト生成結果 {result=}")

        return result

    def prepare_inputs_for_generation(self, input_ids, past_key_values=None, inputs_embeds=None, **kwargs):
        """
        LlamaForCausalLMのgenerateメソッドの内部ループで毎回呼び出される
        LlamaForCausalLM用に入力を整形する
        imagesという引数が正しく渡されるようにオーバーライド

        Args:
            input_ids: テキストのトークンIDのテンソル
            past_key_values: 過去のキーとバリューのテンソル
            inputs_embeds: テキストの入力埋め込みのテンソル
            **kwargs: その他の引数
        Returns:
            整形された入力の辞書
        """
        images = kwargs.pop("images", None)
        image_sizes = kwargs.pop("image_sizes", None)

        logger.info(f"テキスト生成用に入力を整形 {input_ids.shape=}, {past_key_values=}, {inputs_embeds.shape if inputs_embeds is not None else None=}, {images.shape if images is not None else None=}, {image_sizes=}, {kwargs.keys()=}")

        # オーバーライド前のprepare_inputs_for_generationを呼び出す
        inputs = super().prepare_inputs_for_generation(
            input_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            **kwargs
        )

        if images is not None:
            inputs['images'] = images

        if image_sizes is not None:
            inputs['image_sizes'] = image_sizes

        logger.debug(f"整形後の入力 {inputs.keys()=}")

        return inputs

AutoConfig.register("llava_llama", LlavaConfig)
AutoModelForCausalLM.register(LlavaConfig, LlavaLlamaForCausalLM)

### 事前学習済みモデルの読み込み

In [ ]:
def get_model_name_from_path(model_path):
    model_path = model_path.strip("/")
    model_paths = model_path.split("/")
    if model_paths[-1].startswith('checkpoint-'):
        result = model_paths[-2] + "_" + model_paths[-1]
    else:
        result = model_paths[-1]

    logger.info(f"モデル名をパスから抽出 {result=}")
    return result

model_path = "liuhaotian/llava-v1.5-7b"
model_name = get_model_name_from_path(model_path)
model_name

In [ ]:
def load_pretrained_model(model_path, model_base, model_name, load_8bit=False, load_4bit=False, device_map="auto", device="cuda", use_flash_attn=False, **kwargs):
    """
    事前学習済みモデルをロード

    Args:
        model_path: モデルのパス
        model_base: ベースモデルのパス
        model_name: モデル名
        load_8bit: 8ビットでロードするフラグ
        load_4bit: 4ビットでロードするフラグ
        device_map: デバイスマッピング設定
        device: 使用するデバイス
        use_flash_attn: Flash Attentionを使用するフラグ
        **kwargs: その他の引数
    Returns:
        ロードされたモデルのインスタンス
    """

    logger.info(f"事前学習済みモデルをロード {model_path=}, {model_base=}, {model_name=}, {load_8bit=}, {load_4bit=}, {device_map=}, {device=}, {use_flash_attn=}, {kwargs=}")

    kwargs = {"device_map": device_map, **kwargs}

    # False
    if device != "cuda":
        kwargs['device_map'] = {"": device}

    # False
    if load_8bit:
        kwargs['load_in_8bit'] = True

    # False
    elif load_4bit:
        kwargs['load_in_4bit'] = True
        kwargs['quantization_config'] = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type='nf4'
        )

    # True
    else:
        kwargs['torch_dtype'] = torch.float16

    # False
    if use_flash_attn:
        kwargs['attn_implementation'] = 'flash_attention_2'

    # True
    if 'llava' in model_name.lower():

        logger.debug(f"通常のLLaVAモデルをロード")
    
        # False
        if 'lora' in model_name.lower() and model_base is None:
            logger.warning('There is `lora` in model name but no `model_base` is provided. If you are loading a LoRA model, please provide the `model_base` argument. Detailed instruction: https://github.com/haotian-liu/LLaVA#launch-a-model-worker-lora-weights-unmerged.')

        # False
        if 'lora' in model_name.lower() and model_base is not None:
            from llava.model.language_model.llava_llama import LlavaConfig
            lora_cfg_pretrained = LlavaConfig.from_pretrained(model_path)
            tokenizer = AutoTokenizer.from_pretrained(model_base, use_fast=False)
            print('Loading LLaVA from base model...')
            model = LlavaLlamaForCausalLM.from_pretrained(model_base, low_cpu_mem_usage=True, config=lora_cfg_pretrained, **kwargs)
            token_num, tokem_dim = model.lm_head.out_features, model.lm_head.in_features
            if model.lm_head.weight.shape[0] != token_num:
                model.lm_head.weight = torch.nn.Parameter(torch.empty(token_num, tokem_dim, device=model.device, dtype=model.dtype))
                model.model.embed_tokens.weight = torch.nn.Parameter(torch.empty(token_num, tokem_dim, device=model.device, dtype=model.dtype))

            print('Loading additional LLaVA weights...')
            if os.path.exists(os.path.join(model_path, 'non_lora_trainables.bin')):
                non_lora_trainables = torch.load(os.path.join(model_path, 'non_lora_trainables.bin'), map_location='cpu')
            else:
                # this is probably from HF Hub
                from huggingface_hub import hf_hub_download
                def load_from_hf(repo_id, filename, subfolder=None):
                    cache_file = hf_hub_download(
                        repo_id=repo_id,
                        filename=filename,
                        subfolder=subfolder)
                    return torch.load(cache_file, map_location='cpu')
                non_lora_trainables = load_from_hf(model_path, 'non_lora_trainables.bin')
            non_lora_trainables = {(k[11:] if k.startswith('base_model.') else k): v for k, v in non_lora_trainables.items()}
            if any(k.startswith('model.model.') for k in non_lora_trainables):
                non_lora_trainables = {(k[6:] if k.startswith('model.') else k): v for k, v in non_lora_trainables.items()}
            model.load_state_dict(non_lora_trainables, strict=False)

            from peft import PeftModel
            print('Loading LoRA weights...')
            model = PeftModel.from_pretrained(model, model_path)
            print('Merging LoRA weights...')
            model = model.merge_and_unload()
            print('Model is loaded...')

        # False
        elif model_base is not None:
            # this may be mm projector only
            if 'mpt' in model_name.lower():
                if not os.path.isfile(os.path.join(model_path, 'configuration_mpt.py')):
                    shutil.copyfile(os.path.join(model_base, 'configuration_mpt.py'), os.path.join(model_path, 'configuration_mpt.py'))
                tokenizer = AutoTokenizer.from_pretrained(model_base, use_fast=True)
                cfg_pretrained = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
                model = LlavaMptForCausalLM.from_pretrained(model_base, low_cpu_mem_usage=True, config=cfg_pretrained, **kwargs)
            else:
                tokenizer = AutoTokenizer.from_pretrained(
                    model_base,
                    use_fast=False
                )
                cfg_pretrained = AutoConfig.from_pretrained(model_path)
                model = LlavaLlamaForCausalLM.from_pretrained(
                    model_base,
                    low_cpu_mem_usage=True,
                    config=cfg_pretrained,
                    **kwargs)

            mm_projector_weights = torch.load(
                os.path.join(model_path, 'mm_projector.bin'),
                map_location='cpu'
            )

            mm_projector_weights = {
                k: v.to(torch.float16) for k, v in mm_projector_weights.items()
            }

            model.load_state_dict(mm_projector_weights, strict=False)

        # True
        else:

            # False
            if 'mpt' in model_name.lower():
                tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
                model = LlavaMptForCausalLM.from_pretrained(model_path, low_cpu_mem_usage=True, **kwargs)

            # False
            elif 'mistral' in model_name.lower():
                tokenizer = AutoTokenizer.from_pretrained(model_path)
                model = LlavaMistralForCausalLM.from_pretrained(
                    model_path,
                    low_cpu_mem_usage=True,
                    **kwargs
                )

            # True
            else:
                tokenizer = AutoTokenizer.from_pretrained(
                    model_path,
                    use_fast=False
                )
                logger.debug(f"トークナイザーをロード {tokenizer=} {model_path}")

                model = LlavaLlamaForCausalLM.from_pretrained(
                    model_path,
                    low_cpu_mem_usage=True,
                    **kwargs
                )
                logger.debug(f"モデルをロード {model=} {model_path=}")
    else:
        # Load language model

        if model_base is not None:
            # PEFT model
            from peft import PeftModel
            tokenizer = AutoTokenizer.from_pretrained(model_base, use_fast=False)
            model = AutoModelForCausalLM.from_pretrained(model_base, low_cpu_mem_usage=True, **kwargs)
            print(f"Loading LoRA weights from {model_path}")
            model = PeftModel.from_pretrained(model, model_path)
            print(f"Merging weights")
            model = model.merge_and_unload()
            print('Convert to FP16...')
            model.to(torch.float16)
        else:
            use_fast = False
            if 'mpt' in model_name.lower():
                tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
                model = AutoModelForCausalLM.from_pretrained(model_path, low_cpu_mem_usage=True, trust_remote_code=True, **kwargs)
            else:
                tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
                model = AutoModelForCausalLM.from_pretrained(model_path, low_cpu_mem_usage=True, **kwargs)

    # ビジョンタワーのロード

    image_processor = None

    # True
    if 'llava' in model_name.lower():

        # False
        mm_use_im_start_end = getattr(model.config, "mm_use_im_start_end", False)

        # False
        mm_use_im_patch_token = getattr(model.config, "mm_use_im_patch_token", True)

        # False
        if mm_use_im_patch_token:
            tokenizer.add_tokens([DEFAULT_IMAGE_PATCH_TOKEN], special_tokens=True)

        # False
        if mm_use_im_start_end:
            tokenizer.add_tokens([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN], special_tokens=True)

        # トークナイザーの語彙サイズに合わせ入力層と出力層をリサイズ
        # 32000 -> 32000
        # https://huggingface.co/docs/transformers/en/main_classes/model#transformers.PreTrainedModel.resize_token_embeddings
        model.resize_token_embeddings(len(tokenizer))

        vision_tower = model.get_vision_tower()

        # True
        if not vision_tower.is_loaded:

            # ビジョンタワーの重みをロード
            vision_tower.load_model(device_map=device_map)

        # False
        if device_map != 'auto':
            vision_tower.to(device=device_map, dtype=torch.float16)

        # CLIPVisionModel入力の前処理オブジェクト
        image_processor = vision_tower.image_processor

    # コンテキスト長を決定

    # False
    if hasattr(model.config, "max_sequence_length"):
        context_len = model.config.max_sequence_length
    else:
        context_len = 2048

    logger.debug(f"{context_len=}")

    return tokenizer, model, image_processor, context_len

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=model_name
)

### Conversation

In [ ]:
CONTROLLER_HEART_BEAT_EXPIRATION = 30
WORKER_HEART_BEAT_INTERVAL = 15

LOGDIR = "."

# Model Constants
IGNORE_INDEX = -100
IMAGE_TOKEN_INDEX = -200
DEFAULT_IMAGE_TOKEN = "<image>"
DEFAULT_IMAGE_PATCH_TOKEN = "<im_patch>"
DEFAULT_IM_START_TOKEN = "<im_start>"
DEFAULT_IM_END_TOKEN = "<im_end>"
IMAGE_PLACEHOLDER = "<image-placeholder>"


In [ ]:
class SeparatorStyle(Enum):
    """
    チャットフォーマットの区切り方
    """
    SINGLE = auto() # sep="###"
    TWO = auto() # sep=" ", sep2="</s>"
    MPT = auto() 
    PLAIN = auto()
    LLAMA_2 = auto()

In [ ]:
@dataclasses.dataclass
class Conversation:
    """
    チャットフォーマットを管理するクラス
    """

    # システムプロンプト
    system: str

    # ロール（USER, ASSISTANT)
    roles: List[str]

    # [[role, message], ...]
    messages: List[List[str]]

    # メッセージ表示開始位置
    offset: int

    # 区切りスタイル
    sep_style: SeparatorStyle = SeparatorStyle.SINGLE

    # 区切り文字1
    sep: str = "###"

    # 区切り文字2
    sep2: str = None

    # モデルバージョン
    version: str = "Unknown"

    skip_next: bool = False

    def get_prompt(self):
        """
        追加されたメッセージからチャットプロンプトを構築して返す

        Returns:
            構築されたチャットプロンプトの文字列
        例:
            A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.
            ### USER: <image>
            What are the things you can see in this image?
            ### ASSISTANT: This is a photo of a cat sitting on a windowsill looking outside. The cat appears to be a tabby with a mix of brown, black, and white fur. Outside the window, there are green trees and a bright sky, indicating that it is likely daytime.
            ### USER: Can you describe the cat's fur pattern?
            ### ASSISTANT:
        """

        # メッセージのリストを取得
        messages = self.messages

        logger.info(f"プロンプトを構築開始 {messages=}, {self.sep_style=}, {self.version=}")

        # メッセージが画像つきの場合
        # False
        if len(messages) > 0 and type(messages[0][1]) is tuple:
            messages = self.messages.copy()
            init_role, init_msg = messages[0].copy()
            init_msg = init_msg[0].replace("<image>", "").strip()
            if 'mmtag' in self.version:
                messages[0] = (init_role, init_msg)
                messages.insert(0, (self.roles[0], "<Image><image></Image>"))
                messages.insert(1, (self.roles[1], "Received."))
            else:
                messages[0] = (init_role, "<image>\n" + init_msg)

        # False
        if self.sep_style == SeparatorStyle.SINGLE:
            ret = self.system + self.sep
            for role, message in messages:
                if message:
                    if type(message) is tuple:
                        message, _, _ = message
                    ret += role + ": " + message + self.sep
                else:
                    ret += role + ":"

        # True
        elif self.sep_style == SeparatorStyle.TWO:

            # セパレータを取得
            # [" ", "</s>"]
            seps = [self.sep, self.sep2]

            # 戻り値にシステムプロンプトを追加
            # "A chat between a curious human..."" + " "
            ret = self.system + seps[0]

            # メッセージのリストでループ
            for i, (role, message) in enumerate(messages):

                # メッセージが存在する場合
                if message:

                    # メッセージがタプルの場合は抽出
                    if type(message) is tuple:
                        message, _, _ = message

                    # iが偶数: "USER: <image>\nWhat are the thing..." + " "
                    # iが奇数: "ASSISTANT: This is a photo..." + "</s>"
                    ret += role + ": " + message + seps[i % 2]

                # メッセージが存在しない場合（生成対象）
                else:
                    # "ASSISTANT:"を戻り値に追加
                    ret += role + ":"

        # False
        elif self.sep_style == SeparatorStyle.MPT:
            ret = self.system + self.sep
            for role, message in messages:
                if message:
                    if type(message) is tuple:
                        message, _, _ = message
                    ret += role + message + self.sep
                else:
                    ret += role

        # False
        elif self.sep_style == SeparatorStyle.LLAMA_2:
            wrap_sys = lambda msg: f"<<SYS>>\n{msg}\n<</SYS>>\n\n" if len(msg) > 0 else msg
            wrap_inst = lambda msg: f"[INST] {msg} [/INST]"
            ret = ""

            for i, (role, message) in enumerate(messages):
                if i == 0:
                    assert message, "first message should not be none"
                    assert role == self.roles[0], "first message should come from user"
                if message:
                    if type(message) is tuple:
                        message, _, _ = message
                    if i == 0: message = wrap_sys(self.system) + message
                    if i % 2 == 0:
                        message = wrap_inst(message)
                        ret += self.sep + message
                    else:
                        ret += " " + message + " " + self.sep2
                else:
                    ret += ""
            ret = ret.lstrip(self.sep)

        # False
        elif self.sep_style == SeparatorStyle.PLAIN:
            seps = [self.sep, self.sep2]
            ret = self.system
            for i, (role, message) in enumerate(messages):
                if message:
                    if type(message) is tuple:
                        message, _, _ = message
                    ret += message + seps[i % 2]
                else:
                    ret += ""

        # False
        else:
            raise ValueError(f"Invalid style: {self.sep_style}")

        logger.info(f"プロンプトを構築完了 {ret=}")

        return ret

    def append_message(self, role, message):
        """
        Conversationにメッセージを追加する
        eval_model関数から呼び出される

        Args:
            role: メッセージのロール（USER, ASSISTANT)
            message: メッセージの内容
        """

        logger.info(f"メッセージを追加 {role=}, {message=}")
        self.messages.append([role, message])

    def process_image(self, image, image_process_mode, return_pil=False, image_format='PNG', max_len=1344, min_len=672):
        """
        今回は使わない
        """
        if image_process_mode == "Pad":
            def expand2square(pil_img, background_color=(122, 116, 104)):
                width, height = pil_img.size
                if width == height:
                    return pil_img
                elif width > height:
                    result = Image.new(pil_img.mode, (width, width), background_color)
                    result.paste(pil_img, (0, (width - height) // 2))
                    return result
                else:
                    result = Image.new(pil_img.mode, (height, height), background_color)
                    result.paste(pil_img, ((height - width) // 2, 0))
                    return result
            image = expand2square(image)
        elif image_process_mode in ["Default", "Crop"]:
            pass
        elif image_process_mode == "Resize":
            image = image.resize((336, 336))
        else:
            raise ValueError(f"Invalid image_process_mode: {image_process_mode}")
        if max(image.size) > max_len:
            max_hw, min_hw = max(image.size), min(image.size)
            aspect_ratio = max_hw / min_hw
            shortest_edge = int(min(max_len / aspect_ratio, min_len, min_hw))
            longest_edge = int(shortest_edge * aspect_ratio)
            W, H = image.size
            if H > W:
                H, W = longest_edge, shortest_edge
            else:
                H, W = shortest_edge, longest_edge
            image = image.resize((W, H))
        if return_pil:
            return image
        else:
            buffered = BytesIO()
            image.save(buffered, format=image_format)
            img_b64_str = base64.b64encode(buffered.getvalue()).decode()
            return img_b64_str

    def get_images(self, return_pil=False):
        """
        今回は使わない
        """
        images = []
        for i, (role, msg) in enumerate(self.messages[self.offset:]):
            if i % 2 == 0:
                if type(msg) is tuple:
                    msg, image, image_process_mode = msg
                    image = self.process_image(image, image_process_mode, return_pil=return_pil)
                    images.append(image)
        return images

    def to_gradio_chatbot(self):
        """
        今回は使わない
        """
        ret = []
        for i, (role, msg) in enumerate(self.messages[self.offset:]):
            if i % 2 == 0:
                if type(msg) is tuple:
                    msg, image, image_process_mode = msg
                    img_b64_str = self.process_image(
                        image, "Default", return_pil=False,
                        image_format='JPEG')
                    img_str = f'<img src="data:image/jpeg;base64,{img_b64_str}" alt="user upload image" />'
                    msg = img_str + msg.replace('<image>', '').strip()
                    ret.append([msg, None])
                else:
                    ret.append([msg, None])
            else:
                ret[-1][-1] = msg
        return ret

    def copy(self):
        """
        今回は使わない
        """
        return Conversation(
            system=self.system,
            roles=self.roles,
            messages=[[x, y] for x, y in self.messages],
            offset=self.offset,
            sep_style=self.sep_style,
            sep=self.sep,
            sep2=self.sep2,
            version=self.version)

    def dict(self):
        """
        今回は使わない
        """
        if len(self.get_images()) > 0:
            return {
                "system": self.system,
                "roles": self.roles,
                "messages": [[x, y[0] if type(y) is tuple else y] for x, y in self.messages],
                "offset": self.offset,
                "sep": self.sep,
                "sep2": self.sep2,
            }
        return {
            "system": self.system,
            "roles": self.roles,
            "messages": self.messages,
            "offset": self.offset,
            "sep": self.sep,
            "sep2": self.sep2,
        }

In [ ]:
# LLaVA v1 チャットフォーマットの定義

conv_llava_v1 = Conversation(
    system="A chat between a curious human and an artificial intelligence assistant. "
           "The assistant gives helpful, detailed, and polite answers to the human's questions.",
    roles=("USER", "ASSISTANT"),
    version="v1",
    messages=(),
    offset=0,
    sep_style=SeparatorStyle.TWO,
    sep=" ",
    sep2="</s>",
)

In [ ]:
# チャットフォーマットの辞書
# eval_model関数で使用される

conv_templates = {
    # "default": conv_vicuna_v0,
    # "v0": conv_vicuna_v0,
    # "v1": conv_vicuna_v1,
    # "vicuna_v1": conv_vicuna_v1,
    # "llama_2": conv_llama_2,
    # "mistral_instruct": conv_mistral_instruct,
    # "chatml_direct": conv_chatml_direct,
    # "mistral_direct": conv_chatml_direct,

    # "plain": conv_llava_plain,
    # "v0_plain": conv_llava_plain,
    # "llava_v0": conv_llava_v0,
    # "v0_mmtag": conv_llava_v0_mmtag,
    "llava_v1": conv_llava_v1,
    # "v1_mmtag": conv_llava_v1_mmtag,
    # "llava_llama_2": conv_llava_llama_2,

    # "mpt": conv_mpt,
}

### 評価

In [ ]:
def disable_torch_init():
    """
    PyTorchデフォルトの初期化を無効化
    モデル作成を高速化する
    eval_model関数から呼び出される
    """
    setattr(torch.nn.Linear, "reset_parameters", lambda self: None)
    setattr(torch.nn.LayerNorm, "reset_parameters", lambda self: None)
    logger.info("PyTorchデフォルトの初期化を無効化")

In [ ]:
def image_parser(args):
    """
    セパレーター（,など）で区切られた画像ファイルパスをリストとして取得
    eval_modelから呼び出される
    """
    out = args.image_file.split(args.sep)
    logger.info(f"画像ファイルパスを取得 {out=}")
    return out

In [ ]:
def load_image(image_file):
    """
    画像ファイルをPIL画像としてロード
    URLの場合はダウンロードしてからロード
    load_images関数から呼び出される

    Args:
        image_file: 画像ファイルのパスまたはURL
    Returns:
        PIL画像のインスタンス
    """

    # 画像ファイルがURLの場合
    # True
    if image_file.startswith("http") or image_file.startswith("https"):
        # 画像をダウンロードしてPIL画像としてロード
        # https://llava-vl.github.io/static/images/view.jpg
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        image = Image.open(image_file).convert("RGB")

    logger.info(f"PIL画像をロード {image_file=}, {image.size=}, {image.mode=}")
    return image

In [ ]:
def load_images(image_files):
    """
    画像ファイルパスのリストからPIL画像のリストを取得
    eval_model関数から呼び出される

    Args:
        image_files: 画像ファイルパスのリスト
    Returns:
        PIL画像のリスト
    """
    out = []
    for image_file in image_files:
        image = load_image(image_file)
        out.append(image)
    return out

In [ ]:
def process_anyres_image(image, processor, grid_pinpoints):
    """
    今回は使わない

    Process an image with variable resolutions.

    Args:
        image (PIL.Image.Image): The input image to be processed.
        processor: The image processor object.
        grid_pinpoints (str): A string representation of a list of possible resolutions.

    Returns:
        torch.Tensor: A tensor containing the processed image patches.
    """
    if type(grid_pinpoints) is list:
        possible_resolutions = grid_pinpoints
    else:
        possible_resolutions = ast.literal_eval(grid_pinpoints)
    best_resolution = select_best_resolution(image.size, possible_resolutions)
    image_padded = resize_and_pad_image(image, best_resolution)

    patches = divide_to_patches(image_padded, processor.crop_size['height'])

    image_original_resize = image.resize((processor.size['shortest_edge'], processor.size['shortest_edge']))

    image_patches = [image_original_resize] + patches
    image_patches = [processor.preprocess(image_patch, return_tensors='pt')['pixel_values'][0]
                     for image_patch in image_patches]
    return torch.stack(image_patches, dim=0)

In [ ]:
def expand2square(pil_img, background_color):
    """
    PIL画像を正方形に拡張する
    process_images関数から呼び出される

    Args:
        pil_img (PIL.Image.Image): 入力PIL画像
        background_color (tuple): 背景色のRGBタプル
    Returns:
        PIL.Image.Image: 正方形に拡張されたPIL画像
    """
    logger.info(f"PIL画像を正方形に拡張 {pil_img.size=}, {background_color=}")

    # 1000, 667
    width, height = pil_img.size

    # False
    if width == height:
        return pil_img

    # True
    elif width > height:

        # (255, 255, 255)の背景色で正方形の新しい画像を作成
        # 1000x1000
        result = Image.new(pil_img.mode, (width, width), background_color)

        # 中央に元の画像を貼り付け
        result.paste(pil_img, (0, (width - height) // 2))

        return result

    # False
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

In [ ]:
def process_images(images, image_processor, model_cfg):
    """
    PIL画像を前処理し、テンソルに変換する
    eval_model関数から呼び出される

    Args:
        images (list of PIL.Image.Image): 入力PIL画像のリスト
        image_processor: 画像プロセッサーオブジェクト
        model_cfg: モデルの設定オブジェクト
    Returns:
        torch.Tensor: 前処理された画像テンソル
    """
    logger.info(f"画像を前処理 {len(images)=}, {model_cfg.image_aspect_ratio=}")

    # "pad"
    image_aspect_ratio = getattr(model_cfg, "image_aspect_ratio", None)

    new_images = []


    # 正方形にパディングする場合
    # True
    if image_aspect_ratio == 'pad':

        # 画像ごとにループ
        for image in images:

            # 正方形に拡張
            image = expand2square(
                image,
                tuple(int(x*255) for x in image_processor.image_mean) # 背景色
            )

            # 画像を前処理
            image = image_processor.preprocess(
                image,
                return_tensors='pt'
            )['pixel_values'][0]

            new_images.append(image)

    # False
    elif image_aspect_ratio == "anyres":
        for image in images:
            image = process_anyres_image(image, image_processor, model_cfg.image_grid_pinpoints)
            new_images.append(image)
    # False
    else:
        return image_processor(images, return_tensors='pt')['pixel_values']

    # すべての画像テンソルの形状が同じ場合
    if all(x.shape == new_images[0].shape for x in new_images):

        # スタックしてバッチテンソルに変換
        new_images = torch.stack(new_images, dim=0)

    logger.debug(f"前処理された画像テンソル {new_images.shape=}, {new_images.dtype=}")

    return new_images


In [ ]:
def tokenizer_image_token(prompt, tokenizer, image_token_index=IMAGE_TOKEN_INDEX, return_tensors=None):
    """
    画像プレースホルダーを含むプロンプトをトークン化

    promptが例えば以下の場合、プレースホルダーは<image>
    A chat between... USER: <image>\nWhat are the things... ASSISTANT: This ...
    プレースホルダーは-200にトークン化される

    Args:
        prompt (str): プロンプト文字列
        tokenizer: トークナイザーオブジェクト
        image_token_index (int): 画像トークンのインデックス
        return_tensors (str or None): 戻り値の形式 ('pt'など)。Noneの場合はリストで返す。
    Returns:
        list or torch.Tensor: トークン化されたプロンプト
    """

    logger.info(f"画像プレースホルダーを含むプロンプトをトークン化 {prompt=}, {image_token_index=}, {return_tensors=}")

    # 1) プロンプトを<image>で分割し、チャンク化し、トークン化

    prompt_chunks = [tokenizer(chunk).input_ids for chunk in prompt.split('<image>')]
    logger.debug(f"{prompt_chunks=}")

    def insert_separator(X, sep):
        "チャンク間にセパレータ（画像のプレースホルダートークン）を挿入する"
        return [
            ele for sublist in zip(X, [sep]*len(X)) for ele in sublist
        ][:-1]

    # 2) 戻り値の構築

    # 戻り値を初期化
    input_ids = []

    offset = 0

    # プロンプトの最初のチャンクがBOSトークンで始まる場合
    # True
    if len(prompt_chunks) > 0 and \
        len(prompt_chunks[0]) > 0 and \
        prompt_chunks[0][0] == tokenizer.bos_token_id:

        # BOSトークン分のオフセット
        offset = 1

        # BOSトークンを戻り値に追加
        input_ids.append(prompt_chunks[0][0])

    # チャンク間に画像のプレースホルダートークン（-200）を挿入し、チャンクごとにループ
    for x in insert_separator(
        prompt_chunks, [image_token_index] * (offset + 1)
    ):
        # BOSトークンをスキップして戻り値に追加
        input_ids.extend(x[offset:])

    # 3) 戻り値の形式を変換して返す

    # True
    if return_tensors is not None:

        # True
        if return_tensors == 'pt':
            result = torch.tensor(input_ids, dtype=torch.long)
            logger.info(f"画像トークン化結果をテンソルで返す {result.shape=}, {result.dtype=}")
            return result

        raise ValueError(f'Unsupported tensor type: {return_tensors}')

    return input_ids

In [ ]:
def eval_model(args):
    """
    プロンプトと画像からテキストを生成する

    Args:
        args: コマンドライン引数の名前空間
    Returns:
        生成されたテキストの文字列
    """
    logger.info(f"モデル評価を実行 {args.model_path=}, {args.model_base=}, {args.model_name=}, {args.query=}, {args.conv_mode=}, {args.image_file=}, {args.sep=}, {args.temperature=}, {args.top_p=}, {args.num_beams=}, {args.max_new_tokens=}")

    # 1) 事前学習済みモデルをロード

    # モデル初期化を無効化
    disable_torch_init()

    # モデル名を抽出
    # "liuhaotian/llava-v1.5-7b" -> "llava-v1.5-7b"
    model_name = get_model_name_from_path(args.model_path)

    # 事前学習済みのトークナイザー、モデル、画像プロセッサ、コンテキスト長をロード
    tokenizer, model, image_processor, context_len = load_pretrained_model(
        args.model_path, # "liuhaotian/llava-v1.5-7b"
        args.model_base, # None
        model_name, # "llava-v1.5-7b"
    )

    # 2) 画像トークンを挿入

    # テキストクエリ
    # What are the things I should be cautious about when I visit here?
    qs = args.query

    # 画像の特殊トークン
    image_token_se = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN

    # 入力クエリに画像プレースホルダーが含まれる場合
    # False
    if IMAGE_PLACEHOLDER in qs:
        if model.config.mm_use_im_start_end:
            qs = re.sub(IMAGE_PLACEHOLDER, image_token_se, qs)
        else:
            qs = re.sub(IMAGE_PLACEHOLDER, DEFAULT_IMAGE_TOKEN, qs)
    # True
    else:
        # 画像トークンを特殊な開始・終了タグで囲むか
        # False
        if model.config.mm_use_im_start_end:
            qs = image_token_se + "\n" + qs
        else:
            # 画像プレースホルダーをクエリの先頭に追加
            # <image>\nWhat are the things...
            qs = DEFAULT_IMAGE_TOKEN + "\n" + qs

    # 3) 会話モードを決定

    # False
    if "llama-2" in model_name.lower():
        conv_mode = "llava_llama_2"

    # False
    elif "mistral" in model_name.lower():
        conv_mode = "mistral_instruct"

    # False
    elif "v1.6-34b" in model_name.lower():
        conv_mode = "chatml_direct"

    # True
    elif "v1" in model_name.lower():
        # チャットフォーマットをLLaVA v1に設定
        conv_mode = "llava_v1"

    # False
    elif "mpt" in model_name.lower():
        conv_mode = "mpt"

    # False
    else:
        conv_mode = "llava_v0"

    # 対話フォーマットの指定がない場合
    # False
    if args.conv_mode is not None and conv_mode != args.conv_mode:
        print(
            "[WARNING] the auto inferred conversation mode is {}, while `--conv-mode` is {}, using {}".format(
                conv_mode, args.conv_mode, args.conv_mode
            )
        )
    # True
    else:
        args.conv_mode = conv_mode

    logger.debug(f"会話モードを決定 {args.conv_mode=}")

    # 4) プロンプトを構築

    # 会話テンプレートを取得
    conv = conv_templates[args.conv_mode].copy()

    # 会話テンプレートにユーザメッセージを追加
    # USER, <image>\nWhat are the things...
    conv.append_message(conv.roles[0], qs)

    # アシスタントメッセージを追加
    # ASSISTANT, None
    conv.append_message(conv.roles[1], None)

    # 最終的なプロンプトを構築
    prompt = conv.get_prompt()

    # 5) 画像の前処理

    # 画像ファイルパスを取得
    # ["https://llava-vl.github.io/static/images/view.jpg"]
    image_files = image_parser(args)

    # 画像をダウンロードし、PIL画像オブジェクトを取得    
    images = load_images(image_files)

    # 画像サイズを取得
    # [(1000, 667)]
    image_sizes = [x.size for x in images]
    logger.debug(f"画像サイズを取得 {image_sizes=}")

    # 画像を前処理し、テンソルに変換
    images_tensor = process_images(
        images,
        image_processor,
        model.config
    ).to(model.device, dtype=torch.float16)

    # 6) トークン化

    # プロンプトをトークン化
    # このとき、<image>は-200にトークン化される
    input_ids = (
        tokenizer_image_token(
            prompt,
            tokenizer,
            IMAGE_TOKEN_INDEX,
            return_tensors="pt"
        ).unsqueeze(0).cuda()
    )
    logger.debug(f"プロンプトをトークン化 {input_ids}")

    # 7) テキスト生成

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=images_tensor,
            image_sizes=image_sizes,
            do_sample=True if args.temperature > 0 else False,
            temperature=args.temperature,
            top_p=args.top_p,
            num_beams=args.num_beams,
            max_new_tokens=args.max_new_tokens,
            use_cache=True,
        )

    # 8) 結果のデコード

    outputs = tokenizer.batch_decode(
        output_ids,
        skip_special_tokens=True
    )[0].strip()

    logger.info(f"生成結果をデコード完了 {outputs=}")

prompt = "What are the things I should be cautious about when I visit here?"
image_file = "https://llava-vl.github.io/static/images/view.jpg"

args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 512
})()

eval_model(args)